In [2]:
import time, os, glob

import torch
torch.manual_seed(123456789)
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from torchvision import transforms

import numpy as np
import matplotlib
matplotlib.use('PDF')   # for heatmap with original
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from PIL import Image
from tqdm import tqdm
import copy

In [3]:
IMG_DIR  = 'data/norm_tiles/'

In [5]:
SAVE_ORIG_DIR = 'data/outputs/norm_png/'
if not os.path.exists(SAVE_ORIG_DIR):
    os.makedirs(SAVE_ORIG_DIR)
        
filenames = glob.glob(IMG_DIR + '*')
print(filenames)
filenames = [filename.split('/')[-1] for filename in filenames]
print(filenames)

['data/norm_tiles/NA4077-02_AB', 'data/norm_tiles/NA4391-02_AB', 'data/norm_tiles/NA4160-02_AB', 'data/norm_tiles/NA4672-02_AB', 'data/norm_tiles/NA4553-02_AB', 'data/norm_tiles/NA4695-02_AB', 'data/norm_tiles/NA4256-02_AB', 'data/norm_tiles/NA4675-02_AB', 'data/norm_tiles/NA4691-02_AB', 'data/norm_tiles/NA4450-02_AB', 'data/norm_tiles/NA4092-02_AB', 'data/norm_tiles/NA4626-02_AB', 'data/norm_tiles/NA4463-02_AB', 'data/norm_tiles/NA4107-02_AB', 'data/norm_tiles/NA4299-02_AB', 'data/norm_tiles/NA4195-02_AB', 'data/norm_tiles/NA4471-02_AB', 'data/norm_tiles/NA3777-02_AB']
['NA4077-02_AB', 'NA4391-02_AB', 'NA4160-02_AB', 'NA4672-02_AB', 'NA4553-02_AB', 'NA4695-02_AB', 'NA4256-02_AB', 'NA4675-02_AB', 'NA4691-02_AB', 'NA4450-02_AB', 'NA4092-02_AB', 'NA4626-02_AB', 'NA4463-02_AB', 'NA4107-02_AB', 'NA4299-02_AB', 'NA4195-02_AB', 'NA4471-02_AB', 'NA3777-02_AB']


In [3]:
SAVE_ORIG_DIR = 'data/outputs/norm_png/'
if not os.path.exists(SAVE_ORIG_DIR):
    os.makedirs(SAVE_ORIG_DIR)
        
filenames = glob.glob(IMG_DIR + '*')
filenames = [filename.split('/')[-1] for filename in filenames]
img_size = 1536

t = tqdm(total=len(filenames))
for i, filename in enumerate(filenames):
    # Get image dimension
    TILE_DIR = IMG_DIR+'{}/0/'.format(filename)
    imgs = []
    for target in sorted(os.listdir(TILE_DIR)):
        d = os.path.join(TILE_DIR, target)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if fname.endswith('.jpg'):
                    path = os.path.join(root, fname)
                    imgs.append(path)
                    
    rows = [int(image.split('/')[-2]) for image in imgs]
    row_nums = max(rows) + 1
    cols = [int(image.split('/')[-1].split('.')[0]) for image in imgs]
    col_nums = max(cols) + 1
    width = col_nums*img_size
    height = row_nums*img_size
    
    t.set_description_str("Image " + str(i) + ': ({}, {}) ({}, {})'.format(col_nums, row_nums, width, height))
    t.refresh()
    t.write("", end=' ')
    
    # Generate WSI
    orig_img = Image.new('RGB', (width, height))
    for r in range(row_nums):
        for c in range(col_nums):
            img_path = TILE_DIR + '{}/{}.jpg'.format(r, c)
            img = Image.open(img_path)
            orig_img.paste(img, (c*img_size, r*img_size))

    orig_img.save(SAVE_ORIG_DIR+filename+'.png')
    
#     orig_img.save(SAVE_ORIG_DIR+filename+'.jpg')
    
    t.update()
t.close()

Image 0: (48, 31) (73728, 47616):   0%|          | 0/18 [00:00<?, ?it/s]

OSError: encoder error -2 when writing image file

In [5]:
from PIL import Image


def IsValidImage(img_path):
    """
    判断文件是否为有效（完整）的图片
    :param img_path:图片路径
    :return:True：有效 False：无效
    """
    bValid = True
    try:
        Image.open(img_path).verify()
    except:
        bValid = False
    return bValid


def transimg(img_path):
    """
    转换图片格式
    :param img_path:图片路径
    :return: True：成功 False：失败
    """
    if IsValidImage(img_path):
        try:
            str = img_path.rsplit(".", 1)
            output_img_path = str[0] + ".jpg"
            print(output_img_path)
            im = Image.open(img_path)
            im.save(output_img_path)
            return True
        except:
            return False
    else:
        return False


if __name__ == '__main__':
    img_path = 'data/outputs/norm_png/NA4626-02_AB.png'
    print(img_path)
    print(transimg(img_path))


data/outputs/norm_png/NA4626-02_AB.png
False


In [7]:
img_path = 'data/outputs/norm_png/NA4626-02_AB.png'
import cv2
img = cv2.imread(img_path)

error: OpenCV(3.4.5) /io/opencv/modules/imgcodecs/src/loadsave.cpp:75: error: (-215:Assertion failed) pixels <= CV_IO_MAX_IMAGE_PIXELS in function 'validateInputImageSize'


In [5]:
!apt install python-openslide --yes


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-openslide is already the newest version (1.1.0-3build1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [6]:
import openslide


ModuleNotFoundError: No module named 'openslide'